In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv


In [ ]:
load_dotenv()

In [ ]:
url = "https://code4x.dev/courses/chat-app-using-langchain-openai-gpt-api-pinecone-vector-database/"

In [ ]:
loader = WebBaseLoader(url)
documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len,
    is_separator_regex = False
)

split_documents = text_splitter.split_documents(documents)

In [ ]:
print("Totally " + str(len(split_documents)) + " Document(s) Found!")

In [ ]:
for document in split_documents:
    document.page_content = document.page_content.strip().replace("\n\n", "\n")
    
    print(document.page_content)

In [9]:
embeddings = AzureOpenAIEmbeddings(
    model = "text-embedding-ada-002",
    deployment = "text-embedding-ada-002",
    chunk_size=1
)

In [10]:
vector = None

In [14]:
for document in split_documents:
    if(vector is None):
        vector = FAISS.from_documents([document], embeddings)
    else:
        vector.add_documents([document])

In [16]:
llm = AzureChatOpenAI(
    temperature = 0,
    deployment_name = "iomega-gpt-4"
)

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on this provided context:

<context>
{context}
</context>

Question: {input}
""")

In [22]:
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("Setup is Ready!")

Setup is Ready!


In [24]:
response = retrieval_chain.invoke({
    "context": "you are the trainer who is teaching the given course and you are here to suggest to potential learners",
    "input": "what are the key takeaways for learners from this course?"
})

In [25]:
print(response)

{'context': [Document(page_content='In this course, you will learn how...', metadata={'source': 'https://code4x.dev/courses/chat-app-using-langchain-openai-gpt-api-pinecone-vector-database/', 'title': '#1 Best Gen AI Project | LangChain OpenAi GPT Chat app Project for Begineers', 'description': 'Langchain OpenAI Project for Begineers Initiative: Bridging Language and Technology – Find Comprehensive Documentation on Our Project.', 'language': 'en-US'}), Document(page_content='[with full Code and Video Explainers]\nBy the end of this course, you will be able to:', metadata={'source': 'https://code4x.dev/courses/chat-app-using-langchain-openai-gpt-api-pinecone-vector-database/', 'title': '#1 Best Gen AI Project | LangChain OpenAi GPT Chat app Project for Begineers', 'description': 'Langchain OpenAI Project for Begineers Initiative: Bridging Language and Technology – Find Comprehensive Documentation on Our Project.', 'language': 'en-US'}), Document(page_content='Key Components', metadata={